# **Datasets Merging and Normalization**

In [40]:
%load_ext autoreload
%autoreload 2

from utils import *

datasets_folder = './datasets'
verbosity=2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## **Input Data**

We load the datasets with the techniques used in the corresponding notebooks.

- Air pollution

In [41]:
pollution_data = read_and_preprocess_dataset(datasets_folder, 'pollution', v=verbosity)

Stations found: GIARDINI MARGHERITA, PORTA SAN FELICE, VIA CHIARINI
Splitting station "GIARDINI MARGHERITA"...
Splitting station "PORTA SAN FELICE"...
Splitting station "VIA CHIARINI"...


In [42]:
display(pollution_data['GIARDINI MARGHERITA']['NO2'].iloc[:2])
display(pollution_data['GIARDINI MARGHERITA']['NO2'].iloc[-2:])

,Agent_value
Date,
2019-01-02 00:00:00,25.0
2019-01-02 01:00:00,25.0


,Agent_value
Date,
2024-12-31 22:00:00,22.0
2024-12-31 23:00:00,21.0


- Traffic

In [43]:
traffic_data = read_and_preprocess_dataset(datasets_folder, 'traffic', v=verbosity)

Merging readings files...
Merged 6 CSV files
Merging accuracies files...
Merged 6 CSV files
Location GIARDINI MARGHERITA: 44.482671138769533,11.35406170088398
 > Filtering close traffic data...
 > Summing up hour data...
Location PORTA SAN FELICE: 44.499059983334519,11.327526717440112
 > Filtering close traffic data...
 > Summing up hour data...
Location VIA CHIARINI: 44.499134335170289,11.285089594971216
 > Filtering close traffic data...
 > Summing up hour data...


In [44]:
display(traffic_data['GIARDINI MARGHERITA'].iloc[:2])
display(traffic_data['GIARDINI MARGHERITA'].iloc[-2:])

,Traffic_value
Date,
2019-01-01 00:00:00,10501.0
2019-01-01 01:00:00,16863.0


,Traffic_value
Date,
2024-12-31 22:00:00,4162.0
2024-12-31 23:00:00,3765.0


- Weather

In [45]:
weather_data = read_and_preprocess_dataset(datasets_folder, 'weather', v=verbosity)

Merging weather files...
Merged 6 CSV files


In [46]:
display(weather_data.iloc[:2])
display(weather_data.iloc[-2:])

,TAVG,PREC,RHAVG,RAD,W_SCAL_INT,W_VEC_DIR,LEAFW
Date,,,,,,,
2019-01-01 00:00:00,1.0,0.0,92.3,0.0,0.5,208.7,0.0
2019-01-01 01:00:00,0.3,0.0,93.6,0.0,0.5,280.0,0.0


,TAVG,PREC,RHAVG,RAD,W_SCAL_INT,W_VEC_DIR,LEAFW
Date,,,,,,,
2024-12-31 22:00:00,7.0,0.0,77.0,0.0,4.8,232.1,0.0
2024-12-31 23:00:00,7.0,0.0,80.3,0.0,5.3,230.4,0.0


**NOTE:** The very last day of weather data is not present\
**NOTE:** In each dataframe, the Date index represents the starting hour of each measurement. We used it because it was easier to work with.

## **Merge the datasets**

From the preprocessing, we saw that each station has is own data, mainly because of the traffic being different across them, but also because the agent values differ.\
We will then need to create a dataset for each agent-station pair. Some data will be duplicate.

We will merge the dataset traffic, weather and pollution (target values) on the `Date` index.\
If the data to merge is hourly we can simply join the datasets using the index...

**NOTE:** only the process of one station is showed.

In [47]:
merged_giardini_margherita = {}
merged_giardini_margherita['NO2'] = join_datasets(
    pollution_data['GIARDINI MARGHERITA']['NO2'],
    traffic_data['GIARDINI MARGHERITA'],
    weather_data,
    dropna=True # drop the last day (31-12-2024)
    )

In [48]:
merged_giardini_margherita['NO2'].head(3)

,Agent_value,Traffic_value,TAVG,PREC,RHAVG,RAD,W_SCAL_INT,W_VEC_DIR,LEAFW
Date,,,,,,,,,
2019-01-02 00:00:00,25.0,4167.0,0.5,0.0,99.5,0.0,1.4,283.2,1.0
2019-01-02 01:00:00,25.0,1865.0,0.1,0.0,99.7,0.0,1.0,236.6,1.0
2019-01-02 02:00:00,22.0,914.0,0.1,0.0,99.4,0.0,0.6,242.0,0.0


...but if we are considering a daily agent like *PM10*, as introduced in the [traffic notebook](./2-traffic_preprocessing.ipynb) and [weather notebook](./3-weather_preprocessing.ipynb), we should first convert the traffic and weather datasets to represent the daily informations by aggregating the hourly values. Then, we can merge those data with the target values.

In [49]:
transform_traffic_to_daily_df # create bins for the traffic hours

<function utils.preprocessing.transform_traffic_to_daily_df(df: pandas.core.frame.DataFrame, bin_size=0, offset=0, dropna=True, **kwargs)>

In [50]:
transform_weather_to_daily_df # use appropriate aggregation functions for the daily features

<function utils.preprocessing.transform_weather_to_daily_df(df: pandas.core.frame.DataFrame, aggregations={}, max_min_columns=[], keep_splitted_columns=True, **kwargs)>

## **Normalize the features**

The features have different distributions, so we apply the transformation individually for each column. Both scaling to [0, 1], standardization or other functions can be used.

We also save the agents distribution, to be able to go back to the original values.

In [51]:
normalized_giardini_margherita={}
# Applies `MinMaxScaler` to the specified columns, skipping `skip` columns.    
# If no skip columns are specified, all the columns are attempted to be scaled.
# If columns appear in `return_dist`, a dict with their mean and std will be returned.
normalized_giardini_margherita['NO2'], dist = normalize_columns(merged_giardini_margherita['NO2'], return_dists=['Agent_value'], scaler=StandardScaler())

In [52]:
print('Agent value distributions: ', dist['Agent_value'])
normalized_giardini_margherita['NO2'].head(3)

Agent value distributions:  {'mean': 17.136602996327458, 'std': 12.587616590341343}


,Agent_value,Traffic_value,TAVG,PREC,RHAVG,RAD,W_SCAL_INT,W_VEC_DIR,LEAFW
Date,,,,,,,,,
2019-01-02 00:00:00,0.624693,-1.001239,-1.801424,-0.137549,1.678544,-0.663642,-0.919936,0.883229,2.156313
2019-01-02 01:00:00,0.624693,-1.209222,-1.848190,-0.137549,1.688450,-0.663642,-1.140309,0.360257,2.156313
2019-01-02 02:00:00,0.386364,-1.295143,-1.848190,-0.137549,1.673591,-0.663642,-1.360681,0.420859,-0.463754


## **Encode date and time informations**

We also want to encode date and hour informations, to help the models learn that traffic is really small during the night or the weekends, and so on. We could also add a feature for holidays, if needed.

We can (in order of columns required for the encoding method):
- one hot encode the hour/day/month (does not account for day 31 being close to day 1)
- radial basis function (more accurate)(creates a feature for each possible value of the column to encode. For example, it would introduce 12 feature if encoding the months)
- sine/cosine (1 sin feature and 1 cos feature for each column to encode)

The year should not be relevant.

We start by encoding the months using radial and the day using sine/cosine, introducing 12 features + 2 features. Using radial basis for the days would need 31 new columns and might not be needed.

*Source: [here](https://developer.nvidia.com/blog/three-approaches-to-encoding-time-information-as-features-for-ml-models/)*

In [53]:
method = 'radial_months-sin-cos_days_hours'

encoded_giardini_margherita = {}
# Using the given method, introduces new columns to encode the Date index.
encoded_giardini_margherita['NO2'] = encode_date_index(normalized_giardini_margherita['NO2'], method=method)

## **Final Dataset**

What we end up with, is a dataset for each agent of each station, with input data and target values.

*Agent_value* has to be considered as $y_{true}$

In [54]:
encoded_giardini_margherita['NO2'].head(5)

,Agent_value,Traffic_value,TAVG,PREC,RHAVG,RAD,W_SCAL_INT,W_VEC_DIR,LEAFW,hour_sin,...,month_rbf_3,month_rbf_4,month_rbf_5,month_rbf_6,month_rbf_7,month_rbf_8,month_rbf_9,month_rbf_10,month_rbf_11,month_rbf_12
Date,,,,,,,,,,,,,,,,,,,,,
2019-01-02 00:00:00,0.624693,-1.001239,-1.801424,-0.137549,1.678544,-0.663642,-0.919936,0.883229,2.156313,0.000000,...,0.000335,1.522998e-08,1.266417e-14,1.928750e-22,5.380186e-32,1.928750e-22,1.266417e-14,1.522998e-08,0.000335,0.135335
2019-01-02 01:00:00,0.624693,-1.209222,-1.848190,-0.137549,1.688450,-0.663642,-1.140309,0.360257,2.156313,0.258819,...,0.000335,1.522998e-08,1.266417e-14,1.928750e-22,5.380186e-32,1.928750e-22,1.266417e-14,1.522998e-08,0.000335,0.135335
2019-01-02 02:00:00,0.386364,-1.295143,-1.848190,-0.137549,1.673591,-0.663642,-1.360681,0.420859,-0.463754,0.500000,...,0.000335,1.522998e-08,1.266417e-14,1.928750e-22,5.380186e-32,1.928750e-22,1.266417e-14,1.522998e-08,0.000335,0.135335
2019-01-02 03:00:00,0.306920,-1.317188,-1.941722,-0.137549,1.658732,-0.663642,-0.975029,-2.107587,-0.463754,0.707107,...,0.000335,1.522998e-08,1.266417e-14,1.928750e-22,5.380186e-32,1.928750e-22,1.266417e-14,1.522998e-08,0.000335,0.135335
2019-01-02 05:00:00,1.021909,-1.214913,-2.023562,-0.137549,1.658732,-0.663642,-1.360681,0.035925,-0.463754,0.965926,...,0.000335,1.522998e-08,1.266417e-14,1.928750e-22,5.380186e-32,1.928750e-22,1.266417e-14,1.522998e-08,0.000335,0.135335
